In [1]:
import warnings
warnings.filterwarnings("ignore")

from experiment import Exp

import torch
import yaml
import os

In [2]:
### Parameter for training

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   

args.to_save_path     = "../Run_logs"           
args.window_save_path = "../Sliding_window"
args.root_path        = "../datasets"

args.drop_transition  = False
# Nomalization of the data, depends on dataset!
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"

args.batch_size       = 32 # Reduce if not enough graphic memory
args.shuffle          = True # Give NN more variety to learn
args.drop_last        = True # Drop last batch if smaller than batch size
args.train_vali_quote = 0.90 # 90% training, 10% validation                     

# Training setting 
args.train_epochs            = 60

# According to ShaSpec paper
args.learning_rate           = 0.001 # 10^(-3)
args.learning_rate_patience  = 20 # Reduce learning rate every 20 epochs
args.learning_rate_factor    = 0.1 # Reduce learning rate by 10%

args.early_stop_patience     = 15 # Stop after 15 epochs without improvement

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0 # Use first GPU
args.use_multi_gpu           = False # Use multiple GPUs

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

In [3]:
# Get content of folder args.window_save_path
files = os.listdir(args.window_save_path)

In [7]:
### Parameter for data

args.seed                             = 1 # Seed for reproducibility

args.data_name                        = "dsads"

args.pos_select       = None
args.sensor_select    = None

args.exp_mode            = "LOCV"
    
config_file = open('configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
args.num_modalities  =  config["num_modalities"]
args.miss_rates       =  config["miss_rates"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# Input information
# args.c_in            =  config["num_channels"]
# For ShaSpec we want the number of channels per modality
args.c_in    =  config["num_channels"] // args.num_modalities
args.f_in    = 1

In [8]:
### Parameter for the model

args.model_type = "shaspec"

args.miss_rate = args.miss_rates[3]

args.activation = "ReLU"
args.shared_encoder_type = "concatenated"

# Ablation studies
args.ablate_shared_encoder = True
args.ablate_missing_modality_features = False

exp = Exp(args)

Use CPU
================  ShaSpec Model Configuration  ================
Number of total modalities:  5
Selected miss rate:  0.6
Number of available modalities:  2
Ablate shared encoder:  True
Ablate missing modality features:  False
Build the ShaSpec model!
Done!
Parameter: 325909
Set the seed as:  1


In [9]:
exp.model

model_builder(
  (model): ShaSpec(
    (specific_encoders): ModuleList(
      (0-1): 2 x SpecificEncoder(
        (be): BaseEncoder(
          (conv_layers): Sequential(
            (0): Conv2d(1, 64, kernel_size=(5, 1), stride=(2, 1))
            (1): ReLU()
            (2): Conv2d(64, 64, kernel_size=(5, 1), stride=(2, 1))
            (3): ReLU()
            (4): Conv2d(64, 64, kernel_size=(5, 1), stride=(2, 1))
            (5): ReLU()
            (6): Conv2d(64, 64, kernel_size=(5, 1), stride=(2, 1))
            (7): ReLU()
          )
          (sa): SelfAttention(
            (query): Conv1d(64, 8, kernel_size=(1,), stride=(1,), bias=False)
            (key): Conv1d(64, 8, kernel_size=(1,), stride=(1,), bias=False)
            (value): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
          )
          (fc_fusion): Linear(in_features=320, out_features=128, bias=True)
        )
      )
    )
    (missing_modality_feature_generation): MissingModalityFeatureGeneration()
 

In [11]:
# exp.train()